### 환경 설정
- **Google ADK Framework**지만 OpenAI API로 모델 호출 설정
- .env 파일에 `OPENAI_API_KEY` 작성 필수

In [ ]:
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import Agent

openai_model = LiteLlm("openai/gpt-4o-mini")

### Tool 정의
- 간단한 도구 정의

In [ ]:
import datetime
from zoneinfo import ZoneInfo

def get_weather(city: str) -> dict:
    if city.lower() == "seoul":
        return {
            "status": "success",
            "report": "Seoul은 맑고 20도 입니다."
        }
    return {
        "status": "error",
        "error_message": f"{city}의 날씨 정보가 없습니다."
    }
    
def get_current_time(city: str) -> dict:
    if city.lower() == "서울":
        now = datetime.datetime.now(ZoneInfo("Asia/Seoul"))
        return {
            "status": "success", 
            "report": f"서울의 현재시각: {now:%Y-%m-%d %H:%M:%S %Z}"
        }
    return {
        "status": "error",
        "error_message": f"{city}의 타임존 정보를 모릅니다."
    }

### Runner, Session
- Runner: Agent를 실행하는 엔진
    * 실행에 필요한 Agent로 반드시 root_agent를 입력 받아야함
    * `InMemoryRunner`: 데모용 엔진. 프로세스 메모리에 보관하여 바로 실행이 가능 -> 커널 재시작 시 기록은 안녕히..
    * `run_async`: 비동기로 이벤트를 진행
        + `event.content.parts`에 토큰 텍스트, 함수 호출, 함수 결과가 순차적으로 도착
        + `ask()`코루틴이 스트림을 순회하며 로그를 찍고 최종 텍스트만 반환
- Session: Runner가 실행될 때 부여받는 하나의 고유의 스레드
    * 대화 이력, 중간 상태(메모리), 마지막 메시지 id 등을 보관
    * 식별자: `user_id`(누가) + `session_id`(어떤 대화)
        + 같은 사용자라도 주제별로 세션을 여러 개 생성 가능
- **스트리밍 이벤트**
    * `p.function_call`: 특정 인자 값으로 특정 함수를 호출하라고 지시
    * 함수를 실행한 후 결과를 `p.function_response`로 반환
    * 모델이 최종 답변 토큰을 반환하고 `p.text`가 누적되어 `final_text`로 반환

#### root_agent(엔트리 에이전트) 인스턴스
- Runner가 실행할 대상 에이전트가 1개 반드시 필요
    + sub agents를 AgentTool로 붙여서 오케스트레이션
    + sub agent가 하나만 있다 하더라도 맨 앞에서 메시지를 받고 routing, function_call 할 상위 에이전트가 **반드시** 필요
- 단일 에이전트 패턴은 main_agent(root_agent)에 함수형 도구만 연결하여 작성 가능
- 모델, prompt, tool 구성이 바뀌면 root_agent를 새로 생성한 후 runner를 다시 생성하여 사용

In [ ]:
root_agent = Agent(
    name="weather_time_agent",
    model=openai_model,
    description="도시의 시간/날씨 질문에 답하는 도우미",
    instruction=(
        "도시 이름이 오면 적절한 함수를 호출해 한국어로 간결하게 답하세요. "
        "가능하면 한 문장으로 요약해요."
    ),
    tools=[get_weather, get_current_time],
)

In [ ]:
import asyncio
from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent

APP_NAME = "HelloADK"
USER_ID = "p_001"

runner = InMemoryRunner(app_name=APP_NAME, agent=root_agent)

session = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

async def ask(message: str):
    content = UserContent(parts=[Part(text=message)])
    final_text = None
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=content
    ):
        for p in event.content.parts:
            if getattr(p, "function_call", None):
                print(f"[Tool Call] {p.function_call.name}({p.function_call.args})")
            if getattr(p, "function_response", None):
                print(f"[Tool Result] {p.function_response}")
            if getattr(p, "text", None):
                final_text = p.text
    return final_text

In [ ]:
# 시간 물어보기
await ask("서울 현재 시간을 알려줘")

In [ ]:
# 날씨 물어보기
await ask("서울 날씨는 어때?")

### Sub Agent(Agent-as-a-Tool)
- Sub Agent: 특정 하위 작업에 전문화된 LLM Agent를 구성
- AgentTool: 서브 에이전트를 도구처럼 호출할 수 있도록 감싸는 어댑터로, 메인 에이전트가 판단해서 특정 도구가 처리해야한다 판단되면 호출하여 사용
    + 반드시 tools=[**AgentTool()**] 작성
    + **명확한 프롬프트를 작성**
    + 제약을 추가하면 사전에 폭주하는 것을 방지(길이 제한, 금지어, 출력 양식, 톤 등)
    + root_agent에도 **sub_agent에 대한 사용 조건을 명시**하면 호출 정확도가 상승

In [ ]:
from google.adk.agents import Agent
from google.adk.tools.agent_tool import AgentTool
from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent

summarizer_agent = Agent(
    name="summarizer_agent",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    instruction="입력한 한국어 텍스트를 3문장으로 요약해줘."
)

root_agent = Agent(
    name="root_agent_with_sub",
    model=LiteLlm(model="openai/gpt-4o-mini"),
    instruction=(
        "사용자와 대화하며 길거나 정리가 필요한 텍스트가 오면 summarizer_agent 도구를 호출하여 3문장으로 요약 후 반환해."
    ),
    tools=[AgentTool(agent=summarizer_agent)],
)

sub_runner = InMemoryRunner(app_name="adk_sub_llm", agent=root_agent)
sub_session = await sub_runner.session_service.create_session(app_name="adk_sub_llm", user_id="p_002")

async def ask_sub(message: str):
    content = UserContent(parts=[Part(text=message)])
    final_text = None
    
    async for event in sub_runner.run_async(
        user_id=sub_session.user_id,
        session_id=sub_session.id,
        new_message=content
    ):
        for p in event.content.parts:
            if p.function_call:
                print(f"[SubAgent Call] {p.function_call.name}({p.function_call.args})")
            if p.function_response:
                print(f"[SubAgent Result] {p.function_response}")
            if p.text:
                final_text = p.text
    return final_text

In [ ]:
# 긴 텍스트 요약 테스트
long_text = """
Google ADK(Agent Development Kit)는 멀티에이전트 애플리케이션을 빠르게 구성하도록 돕는 개발 키트입니다.
핵심 아이디어는 ‘작고 명확한 역할의 에이전트’를 여러 개 만든 다음, 이들을 도구처럼 연결하여 복잡한 작업을 단계적으로 해결하는 것입니다.
예를 들어, 메인 에이전트는 사용자 질의를 해석하고, 텍스트가 지나치게 길거나 구조화가 필요하면 요약 전담 서브에이전트를 호출하도록 설계합니다.
이때 서브에이전트는 가능한 한 간단하고 예측 가능한 출력을 내도록 규칙을 좁혀야 합니다. (예: 3문장 요약, 첫 문장에 전체 요지)

실무에서 에이전트 시스템의 성패는 ‘정확한 라우팅’과 ‘관측성(Observability)’에 달려 있습니다.
툴 호출 여부, 호출 인자, 호출 결과를 모두 추적할 수 있어야 품질을 높이고 디버깅 시간을 줄일 수 있습니다.
스트리밍 로그를 통해 모델이 언제 함수를 부르는지, 그 결과를 어떻게 재구성하는지 살피면 프롬프트 수정 포인트가 명확해집니다.
또한 도메인 지식이 필요한 경우, RAG(Retrieval-Augmented Generation) 같은 검색·지식 보강 단계를 사이에 끼워 넣을 수 있습니다.
이때는 임베딩 품질, 청크 전략, 메타데이터 필터링, 인덱스 스케줄링 등 운영적 요소가 품질과 비용에 큰 영향을 줍니다.

비용과 지연(latency)을 줄이기 위한 기본 전략은 ‘작업을 잘게 나눈 뒤, 맞춤형 모델을 배치’하는 것입니다.
가벼운 의사결정·요약·정규화에는 경량 모델을, 정밀한 추론이 필요한 구간에만 고성능 모델을 쓰면 전체 체감 속도가 빨라지고 비용도 절감됩니다.
또한 서브에이전트의 출력 형식을 제한하면, 후속 단계(예: 테이블 생성, 키포인트 추출, 사실 검증)에서 오류를 크게 줄일 수 있습니다.
반대로, 자유도가 너무 큰 프롬프트는 매번 다른 출력을 만들어 파이프라인의 안정성을 해칩니다.

데이터 품질은 언제나 가장 중요한 변수입니다.
잘 정리된 예시(Input-Output 페어)와 부정 예시(원하지 않는 출력)를 함께 제공하면 모델이 따를 규칙을 더 잘 학습합니다.
운영 단계에서는 실패 케이스를 수집해 주기적으로 프롬프트나 시스템 지침을 개선하는 루프를 돌려야 합니다.
이때 실험 추적(버전 관리), A/B 테스트, 샘플 기반 회귀(regression) 검증을 체계화하면 ‘좋아졌는지’를 수치로 확인할 수 있습니다.

마지막으로, 멀티에이전트는 복잡성을 키울 가능성도 큽니다.
서브에이전트가 늘어날수록 라우팅 규칙이 명확해야 하고, 각 에이전트의 입력/출력 계약(Contract)을 문서화해두는 것이 필요합니다.
초기에는 메인+요약 정도의 단순 구조로 시작해, 번역·추출·계획 등 모듈을 점진적으로 추가하세요.
이렇게 하면 교육 참여자들이 흐름을 빠르게 이해하고, 실전 프로젝트로 넘어갈 때도 유지보수 가능한 형태를 유지할 수 있습니다.
"""

await ask_sub(f"아래 글을 3문장으로 요약해줘:\n{long_text}")

### 세션 목록 확인

In [ ]:
sessions = await sub_runner.session_service.list_sessions(
    app_name="adk_sub_llm", user_id="p_002"
)

for s in getattr(sessions, "sessions", []):
    print("id:", getattr(s, "id", None), "\n"
          "user_id:", getattr(s, "user_id", None), "\n"
          "created_at:", getattr(s, "created_at", None))
